## Momentum Strategy

Importing dependencies

In [48]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

Fetching the data source and converting to pandas dataframe

In [49]:
data = pd.read_csv(filepath_or_buffer='../../resources/intraday.csv', parse_dates=['time'], index_col='time')

Getting details about the asset data by analysing the dataframe

In [50]:
data.head(10)

,Close
time,
2009-12-31,1.432706
2010-01-01,1.438994
2010-01-04,1.442398
2010-01-05,1.436596
2010-01-06,1.440403
2010-01-07,1.431803
2010-01-08,1.441109
2010-01-11,1.451126
2010-01-12,1.447660


In [51]:
data.tail(10)

,Close
time,
2021-12-20,1.124354
2021-12-21,1.127752
2021-12-22,1.128757
2021-12-23,1.132888
2021-12-24,1.132734
2021-12-27,1.132426
2021-12-28,1.133003
2021-12-29,1.131478
2021-12-30,1.136015


In [52]:
data.Close.to_frame()

,Close
time,
2009-12-31,1.432706
2010-01-01,1.438994
2010-01-04,1.442398
2010-01-05,1.436596
2010-01-06,1.440403
...,...
2021-12-27,1.132426
2021-12-28,1.133003
2021-12-29,1.131478


Getting more statistical insight into the dataset

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3127 entries, 2009-12-31 to 2021-12-31
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   3127 non-null   float64
dtypes: float64(1)
memory usage: 48.9 KB


In [54]:
data.describe()

,Close
count,3127.000000
mean,1.219630
std,0.107936
min,1.039047
25%,1.124366
50%,1.190079
75%,1.317046
max,1.484406


Since the only attribute we need is closing price
we modify the data frame

In [55]:
data = data.Close.to_frame()

In [56]:
data.head(10)

,Close
time,
2009-12-31,1.432706
2010-01-01,1.438994
2010-01-04,1.442398
2010-01-05,1.436596
2010-01-06,1.440403
2010-01-07,1.431803
2010-01-08,1.441109
2010-01-11,1.451126
2010-01-12,1.447660


We plot the closing price of our asset

In [57]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.Close, name='Close'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [58]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.loc['2018'].index, y=data.loc['2018'].Close, name='Close'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

We proceed to define baseline and compute returns !needs improvement!

In [59]:
data['returns'] = np.log(data.div(data.shift(1)))

In [60]:
data.dropna(inplace=True)

In [61]:
data

,Close,returns
time,,
2010-01-01,1.438994,0.004379
2010-01-04,1.442398,0.002363
2010-01-05,1.436596,-0.004031
2010-01-06,1.440403,0.002647
2010-01-07,1.431803,-0.005989
...,...,...
2021-12-27,1.132426,-0.000272
2021-12-28,1.133003,0.000510
2021-12-29,1.131478,-0.001347


Defining a simple contrarian strategy (window = 3)

In [62]:
window = 3

In [63]:
data['returns'].rolling(window).mean()

time
2010-01-01         NaN
2010-01-04         NaN
2010-01-05    0.000904
2010-01-06    0.000326
2010-01-07   -0.002457
                ...   
2021-12-27    0.001082
2021-12-28    0.000034
2021-12-29   -0.000370
2021-12-30    0.001055
2021-12-31   -0.000147
Name: returns, Length: 3126, dtype: float64

In [64]:
data['position'] = np.sign(data['returns'].rolling(window).mean())

In [65]:
data

,Close,returns,position
time,,,
2010-01-01,1.438994,0.004379,NaN
2010-01-04,1.442398,0.002363,NaN
2010-01-05,1.436596,-0.004031,1.0
2010-01-06,1.440403,0.002647,1.0
2010-01-07,1.431803,-0.005989,-1.0
...,...,...,...
2021-12-27,1.132426,-0.000272,1.0
2021-12-28,1.133003,0.000510,1.0
2021-12-29,1.131478,-0.001347,-1.0


In [66]:
data['strategy'] = data.position.shift(1) * data['returns']

In [67]:
data.dropna(inplace=True)

In [68]:
data

,Close,returns,position,strategy
time,,,,
2010-01-06,1.440403,0.002647,1.0,0.002647
2010-01-07,1.431803,-0.005989,-1.0,-0.005989
2010-01-08,1.441109,0.006478,1.0,-0.006478
2010-01-11,1.451126,0.006927,1.0,0.006927
2010-01-12,1.447660,-0.002391,1.0,-0.002391
...,...,...,...,...
2021-12-27,1.132426,-0.000272,1.0,-0.000272
2021-12-28,1.133003,0.000510,1.0,0.000510
2021-12-29,1.131478,-0.001347,-1.0,-0.001347


In [69]:
data[['returns', 'strategy']].sum()

returns    -0.237846
strategy    0.036154
dtype: float64

In [70]:
data['creturns'] = data['returns'].cumsum().apply(np.exp)
data['cstrategy'] = data['strategy'].cumsum().apply(np.exp)

In [71]:
data

,Close,returns,position,strategy,creturns,cstrategy
time,,,,,,
2010-01-06,1.440403,0.002647,1.0,0.002647,1.002650,1.002650
2010-01-07,1.431803,-0.005989,-1.0,-0.005989,0.996664,0.996664
2010-01-08,1.441109,0.006478,1.0,-0.006478,1.003142,0.990228
2010-01-11,1.451126,0.006927,1.0,0.006927,1.010114,0.997111
2010-01-12,1.447660,-0.002391,1.0,-0.002391,1.007702,0.994729
...,...,...,...,...,...,...
2021-12-27,1.132426,-0.000272,1.0,-0.000272,0.788270,1.045076
2021-12-28,1.133003,0.000510,1.0,0.000510,0.788672,1.045609
2021-12-29,1.131478,-0.001347,-1.0,-0.001347,0.787610,1.044201


In [72]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.creturns, name='Returns (Baseline)'))
fig.add_trace(go.Scatter(x=data.index, y=data.cstrategy, name='Returns (Strategy)'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

Taking trading costs into consideration

In [73]:
ptc = 0.00007

In [74]:
data['trades'] = data.position.diff().fillna(0).abs()

In [75]:
data

,Close,returns,position,strategy,creturns,cstrategy,trades
time,,,,,,,
2010-01-06,1.440403,0.002647,1.0,0.002647,1.002650,1.002650,0.0
2010-01-07,1.431803,-0.005989,-1.0,-0.005989,0.996664,0.996664,2.0
2010-01-08,1.441109,0.006478,1.0,-0.006478,1.003142,0.990228,2.0
2010-01-11,1.451126,0.006927,1.0,0.006927,1.010114,0.997111,0.0
2010-01-12,1.447660,-0.002391,1.0,-0.002391,1.007702,0.994729,0.0
...,...,...,...,...,...,...,...
2021-12-27,1.132426,-0.000272,1.0,-0.000272,0.788270,1.045076,0.0
2021-12-28,1.133003,0.000510,1.0,0.000510,0.788672,1.045609,0.0
2021-12-29,1.131478,-0.001347,-1.0,-0.001347,0.787610,1.044201,2.0


In [76]:
data.trades.value_counts()

0.0    2306
2.0     817
Name: trades, dtype: int64

In [77]:
data['strategy_net'] = data.strategy - data.trades * ptc

In [78]:
data['cstrategy_net'] = data.strategy_net.cumsum().apply(np.exp)

In [79]:
data

,Close,returns,position,strategy,creturns,cstrategy,trades,strategy_net,cstrategy_net
time,,,,,,,,,
2010-01-06,1.440403,0.002647,1.0,0.002647,1.002650,1.002650,0.0,0.002647,1.002650
2010-01-07,1.431803,-0.005989,-1.0,-0.005989,0.996664,0.996664,2.0,-0.006129,0.996524
2010-01-08,1.441109,0.006478,1.0,-0.006478,1.003142,0.990228,2.0,-0.006618,0.989951
2010-01-11,1.451126,0.006927,1.0,0.006927,1.010114,0.997111,0.0,0.006927,0.996832
2010-01-12,1.447660,-0.002391,1.0,-0.002391,1.007702,0.994729,0.0,-0.002391,0.994451
...,...,...,...,...,...,...,...,...,...
2021-12-27,1.132426,-0.000272,1.0,-0.000272,0.788270,1.045076,0.0,-0.000272,0.932515
2021-12-28,1.133003,0.000510,1.0,0.000510,0.788672,1.045609,0.0,0.000510,0.932990
2021-12-29,1.131478,-0.001347,-1.0,-0.001347,0.787610,1.044201,2.0,-0.001487,0.931604


We plot returns with and without trading costs

In [80]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y=data.creturns, name='Returns (Baseline)'))
fig.add_trace(go.Scatter(x=data.index, y=data.cstrategy, name='Returns (Strategy)'))
fig.add_trace(go.Scatter(x=data.index, y=data.cstrategy_net, name='Returns (Strategy + trading costs)'))

fig.update_layout(title='EUR/USD', xaxis_title='Time', yaxis_title='Price')

fig.show()

In [81]:
data[['returns', 'strategy_net']].mean() * (252)

returns        -0.019192
strategy_net   -0.006312
dtype: float64

In [82]:
data[['returns', 'strategy_net']].std() * np.sqrt(252)

returns         0.086889
strategy_net    0.087388
dtype: float64

Now we make use of a comprehensive backtester which implements Contrarian startegy and optimize our parameters to maximize returns

In [83]:
from MomentumContrarianBacktester import MomentumContrarianBacktester as MCB

In [84]:
testMCB = MCB(symbol='EUR/USD', start='2010', end='2021', tc=0.00007, flag='m')

In [85]:
testMCB

MomentumContrarianBacktester(symbol = EUR/USD, start = 2010, end = 2021, flag = m)

In [86]:
testMCB.data

,Close,returns
time,,
2010-01-04,1.442398,0.002363
2010-01-05,1.436596,-0.004031
2010-01-06,1.440403,0.002647
2010-01-07,1.431803,-0.005989
2010-01-08,1.441109,0.006478
...,...,...
2021-12-27,1.132426,-0.000272
2021-12-28,1.133003,0.000510
2021-12-29,1.131478,-0.001347


In [87]:
testMCB.test_strategy()

(0.558864, -0.226289)

In [88]:
testMCB.results

,Close,returns,position,strategy,trades,strategy_net,creturns,cstrategy,cstrategy_net
time,,,,,,,,,
2010-01-05,1.436596,-0.004031,-1.0,-0.004031,0.0,-0.004031,0.995978,0.995978,0.995978
2010-01-06,1.440403,0.002647,1.0,-0.002647,2.0,-0.002787,0.998617,0.993345,0.993206
2010-01-07,1.431803,-0.005989,-1.0,-0.005989,2.0,-0.006129,0.992655,0.987414,0.987138
2010-01-08,1.441109,0.006478,1.0,-0.006478,2.0,-0.006618,0.999107,0.981038,0.980626
2010-01-11,1.451126,0.006927,1.0,0.006927,0.0,0.006927,1.006051,0.987857,0.987442
...,...,...,...,...,...,...,...,...,...
2021-12-27,1.132426,-0.000272,-1.0,0.000272,0.0,0.000272,0.785100,0.563892,0.450034
2021-12-28,1.133003,0.000510,1.0,-0.000510,2.0,-0.000650,0.785500,0.563604,0.449741
2021-12-29,1.131478,-0.001347,-1.0,-0.001347,2.0,-0.001487,0.784442,0.562846,0.449073


We find the number of trades made

In [89]:
testMCB.results.trades.value_counts()

2.0    1597
0.0    1491
1.0      36
Name: trades, dtype: int64

In [90]:
testMCB.plot_results()

We give a set of boundaries to optimize our parameters within

window_range describes the range of days over which need to be taken into consideration

In [91]:
window_range = (1, 200)

In [92]:
testMCB.optimize_parameter(window_range=window_range)

(109, 1.431417)

In [93]:
testMCB.test_strategy(window=4)

(0.817873, 0.026911)

Set of parameters which give maximum returns and a plot of returns for the same

In [94]:
testMCB.plot_results()